## Lab6-Assignment: Topic Classification

Use the same training, development, and test partitions of the the 20 newsgroups text dataset as in Lab6.4-Topic-classification-BERT.ipynb 

* Fine-tune and examine the performance of another transformer-based pretrained language models, e.g., RoBERTa, XLNet

* Compare the performance of this model to the results achieved in Lab6.4-Topic-classification-BERT.ipynb and to a conventional machine learning approach (e.g., SVM, Naive Bayes) using bag-of-words or other engineered features of your choice. 
Describe the differences in performance in terms of Precision, Recall, and F1-score evaluation metrics.

In [1]:
!nvidia-smi

Tue Mar 14 11:17:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install simpletransformers
import nltk
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import spacy
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from simpletransformers.classification import ClassificationModel, ClassificationArgs

nlp = spacy.load('en_core_web_sm')

STOPWORDS = stopwords.words('english')
PUNCT = ['SPACE', 'PUNCT']

### Loading dataset

In [4]:
categories = ['alt.atheism', 'comp.graphics', 'sci.med', 'sci.space'] 

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=categories, random_state=420)
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=categories, random_state=420)

train = pd.DataFrame({'text': newsgroups_train.data, 'labels': newsgroups_train.target})
train, val = train_test_split(train, test_size=0.1, random_state=420, stratify=train[['labels']])
test = pd.DataFrame({'text': newsgroups_test.data, 'labels': newsgroups_test.target})

### Pretrained language model

In [5]:
model_args = ClassificationArgs()

model_args.overwrite_output_dir=True
model_args.evaluate_during_training=True

model_args.num_train_epochs=10
model_args.train_batch_size=16
model_args.learning_rate=4e-6
model_args.max_seq_length=256

model_args.use_early_stopping=True
model_args.early_stopping_delta=0.01 # "The improvement over best_eval_loss necessary to count as a better checkpoint"
model_args.early_stopping_metric='eval_loss'
model_args.early_stopping_metric_minimize=True
model_args.early_stopping_patience=2
model_args.evaluate_during_training_steps=32 # how often you want to run validation in terms of training steps (or batches)

In [6]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=4, args=model_args, use_cuda=True)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [7]:
model.train_model(train, eval_df=val)

  0%|          | 0/2025 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/226 [00:00<?, ?it/s]

  0%|          | 0/226 [00:00<?, ?it/s]

...

  0%|          | 0/226 [00:00<?, ?it/s]

Running Epoch 4 of 10:   0%|          | 0/127 [00:00<?, ?it/s]

  0%|          | 0/226 [00:00<?, ?it/s]

  0%|          | 0/226 [00:00<?, ?it/s]

(544,
 defaultdict(list,
             {'global_step': [32,
               64,
               96,
               127,
               128,
               160,
               192,
               224,
               254,
               256,
               288,
               320,
               352,
               381,
               384,
               416,
               448,
               480,
               508,
               512,
               544],
              'train_loss': [1.39556884765625,
               1.38446044921875,
               1.3515625,
               1.2040473222732544,
               1.155242919921875,
               0.6244430541992188,
               0.4703788757324219,
               0.5655632019042969,
               0.2850341796875,
               0.22103500366210938,
               0.2677440643310547,
               0.24285507202148438,
               0.26853179931640625,
               0.4568040668964386,
               0.2604036331176758,
               0.

In [8]:
print(classification_report(test.labels, model.predict(test.text.to_list())[0]))

  0%|          | 0/1498 [00:00<?, ?it/s]

  0%|          | 0/188 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.84      0.78      0.81       319
           1       0.88      0.93      0.90       389
           2       0.95      0.85      0.90       396
           3       0.77      0.85      0.81       394

    accuracy                           0.86      1498
   macro avg       0.86      0.85      0.85      1498
weighted avg       0.86      0.86      0.86      1498



*Comparison with 'bert'.*

### Conventional ML model

In [9]:
corpus = newsgroups_train.data + newsgroups_test.data
corpus = [[{'word': tok.lemma_, 'pos': tok.pos_}
           for tok in nlp(sent)
           if tok.pos_ not in PUNCT and
           tok.lemma_.lower() not in STOPWORDS]
          for sent in corpus]

vectorizer = DictVectorizer().fit([item for doc in corpus for item in doc])
data = [vectorizer.transform(doc).A.sum(axis=0)
        if doc != []
        else np.zeros(vectorizer.get_feature_names_out().shape[0])
        for doc in corpus]
train, test = data[:len(newsgroups_train.data)], data[-len(newsgroups_test.data):]

In [10]:
vectorizer = CountVectorizer(stop_words=STOPWORDS).fit(newsgroups_train.data + newsgroups_test.data)

train = vectorizer.transform(newsgroups_train.data)
test = vectorizer.transform(newsgroups_test.data)

In [12]:
clf = LinearSVC(C=0.01, max_iter=int(1e6), random_state=420)
clf.fit(train, newsgroups_train.target)
print(classification_report(newsgroups_test.target, clf.predict(test), target_names=newsgroups_test.target_names))

               precision    recall  f1-score   support

  alt.atheism       0.83      0.76      0.80       319
comp.graphics       0.75      0.90      0.82       389
      sci.med       0.87      0.73      0.80       396
    sci.space       0.78      0.79      0.79       394

     accuracy                           0.80      1498
    macro avg       0.81      0.80      0.80      1498
 weighted avg       0.81      0.80      0.80      1498



*Comparison with the other two.*